<a href="https://colab.research.google.com/github/fbelleau/kibio-bi/blob/main/Demo_case_2_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Eland librairy compatible with actual Kibio.science version

In [1]:
!pip install eland==7.14.0b1

import eland as ed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 KB 25.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


## Install most recent version of Pandas (default on Collab is 1.3.5)

In [2]:


import pandas as pd

pd.__version__

'1.3.5'

## Import Elasticsearch's needed librairies

In [3]:
from elasticsearch import Elasticsearch, helpers
import matplotlib.pyplot as plt

## Create connection to Kibio.science's Elastiseach REST endpoint

In [4]:
es = Elasticsearch("http://es.kibio.science:80", verify_certs=False, ssl_show_warn=False)
es

<Elasticsearch([{'host': 'es.kibio.science', 'port': 80}])>

## Dowload needed dataframes from Kibio.science using Elasticsearch's Eland librairy

In [5]:
disease = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_disease_201811'))
print("disease", disease.shape)
gene = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_gene_201811'))
print("gene", gene.shape)
compound = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_compound_201811'))
print("compound", compound.shape)
biological_process = ed.eland_to_pandas(ed.DataFrame(es_client= es, es_index_pattern='hetionet_biological_process_201811'))
print("biological_process", biological_process.shape)

disease (137, 17)
gene (20945, 17)
compound (1552, 20)
biological_process (11381, 11)


In [9]:
disease_association = disease[['@id','associates.@id']]
disease_association_edges = disease_association.explode('associates.@id')
disease_association_edges

,@id,associates.@id
DOID:363,DOID:363,ncbigene:4436
DOID:363,DOID:363,ncbigene:7869
DOID:363,DOID:363,ncbigene:3480
DOID:363,DOID:363,ncbigene:332
DOID:363,DOID:363,ncbigene:3576
...,...,...
DOID:13189,DOID:13189,ncbigene:3557
DOID:13189,DOID:13189,ncbigene:5634
DOID:13189,DOID:13189,ncbigene:55867
DOID:13189,DOID:13189,ncbigene:6568


In [ ]:
disease_association = disease[['@id','associates.@id']]
disease_association_edges = disease_association.explode('associates.@id')
disease_association_edges

In [12]:
# edges_gene_regulates

gene_regulates = gene[['@id','regulates.@id']]
gene_regulates_edges = gene_regulates.explode('regulates.@id')
gene_regulates_edges

,@id,regulates.@id
ncbigene:10670,ncbigene:10670,ncbigene:3927
ncbigene:10670,ncbigene:10670,ncbigene:10417
ncbigene:10670,ncbigene:10670,ncbigene:9552
ncbigene:10670,ncbigene:10670,ncbigene:1022
ncbigene:10670,ncbigene:10670,ncbigene:9917
...,...,...
ncbigene:7124,ncbigene:7124,ncbigene:9093
ncbigene:7124,ncbigene:7124,ncbigene:55450
ncbigene:7124,ncbigene:7124,ncbigene:4792
ncbigene:7124,ncbigene:7124,ncbigene:25966


In [11]:
gene

,@context,@id,@label,@namespace,@type,covaries.@id,data.chromosome,data.description,data.license,data.source,data.url,interacts.@id,kind,name,ncbigene.@id,participates.@id,regulates.@id
ncbigene:10670,http://schema.org/,ncbigene:10670,RRAGA [ncbigene:10670],ncbigene,hetionet:Gene,"[ncbigene:6774, ncbigene:79228, ncbigene:11644...",9,Ras-related GTP binding A,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/10670,"[ncbigene:55035, ncbigene:58528, ncbigene:1091...",Gene,RRAGA,ncbigene:10670,"[GO:0044419, GO:0034198, GO:0035859, GO:190253...","[ncbigene:3927, ncbigene:10417, ncbigene:9552,..."
ncbigene:4880,http://schema.org/,ncbigene:4880,NPPC [ncbigene:4880],ncbigene,hetionet:Gene,"[ncbigene:6038, ncbigene:57333, ncbigene:80757...",2,natriuretic peptide C,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/4880,"[ncbigene:4883, ncbigene:4882]",Gene,NPPC,ncbigene:4880,"[GO:0008285, GO:0019637, GO:0046390, GO:004547...",NaN
ncbigene:125981,http://schema.org/,ncbigene:125981,ACER1 [ncbigene:125981],ncbigene,hetionet:Gene,ncbigene:114971,19,alkaline ceramidase 1,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/125981,NaN,Gene,ACER1,ncbigene:125981,"[GO:0071248, GO:1901565, GO:0008544, GO:004617...",NaN
ncbigene:283554,http://schema.org/,ncbigene:283554,GPR137C [ncbigene:283554],ncbigene,hetionet:Gene,"[ncbigene:374407, ncbigene:256472]",14,G protein-coupled receptor 137C,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/283554,NaN,Gene,GPR137C,ncbigene:283554,NaN,NaN
ncbigene:100289678,http://schema.org/,ncbigene:100289678,ZNF783 [ncbigene:100289678],ncbigene,hetionet:Gene,"[ncbigene:57595, ncbigene:387263, ncbigene:255...",7,zinc finger family member 783,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/100289678,NaN,Gene,ZNF783,ncbigene:100289678,"[GO:0000988, GO:0000989, GO:0003712]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ncbigene:65065,http://schema.org/,ncbigene:65065,NBEAL1 [ncbigene:65065],ncbigene,hetionet:Gene,ncbigene:8744,2,neurobeachin-like 1,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/65065,NaN,Gene,NBEAL1,ncbigene:65065,"[GO:0005543, GO:0019898, GO:0008289]",NaN
ncbigene:2577,http://schema.org/,ncbigene:2577,GAGE5 [ncbigene:2577],ncbigene,hetionet:Gene,NaN,X,G antigen 5,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/2577,"[ncbigene:54386, ncbigene:64395]",Gene,GAGE5,ncbigene:2577,NaN,NaN
ncbigene:79175,http://schema.org/,ncbigene:79175,ZNF343 [ncbigene:79175],ncbigene,hetionet:Gene,NaN,20,zinc finger protein 343,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/79175,NaN,Gene,ZNF343,ncbigene:79175,"[pathwaycommons:PC7_4112, pathwaycommons:PC7_4...",NaN
ncbigene:7124,http://schema.org/,ncbigene:7124,TNF [ncbigene:7124],ncbigene,hetionet:Gene,ncbigene:79290,6,tumor necrosis factor,CC0 1.0,Entrez Gene,http://identifiers.org/ncbigene/7124,"[ncbigene:7334, ncbigene:9919, ncbigene:10569,...",Gene,TNF,ncbigene:7124,"[GO:1901888, GO:0043066, GO:1904950, GO:005179...","[ncbigene:9133, ncbigene:7422, ncbigene:6253, ..."


## Question 1

In [17]:
gene_regulates_gene = gene_regulates_edges[['@id', 'regulates.@id']].groupby(['@id'],as_index=False).agg({'regulates.@id': lambda x: x.tolist()})
gene_regulates_gene

,@id,regulates.@id
0,ncbigene:1,[nan]
1,ncbigene:10,[nan]
2,ncbigene:100,"[ncbigene:1647, ncbigene:9531, ncbigene:5111, ..."
3,ncbigene:1000,[nan]
4,ncbigene:10000,"[ncbigene:5782, ncbigene:302, ncbigene:5111, n..."
...,...,...
20940,ncbigene:9991,[nan]
20941,ncbigene:9992,[nan]
20942,ncbigene:9993,[nan]
20943,ncbigene:9994,[nan]


In [19]:
answer_question_1 = gene_regulates_gene[gene_regulates_gene['regulates.@id'].apply(len) > 15]

answer_question_1.shape

(3599, 2)

## Question 2

In [ ]:
compound_gene = pd.merge(compound[['@id', 'name']], edges_compound_binds[['@id','@id_to']], on='@id')
compound_gene.columns = ['compound_id', 'compound_name', 'gene_id']

compound_gene_disease = pd.merge(compound_gene, edges_disease_associates[['@id', '@id_to']], left_on='gene_id', right_on='@id_to').drop(['@id_to'], axis=1)
compound_gene_disease.columns = ['compound_id', 'compound_name', 'gene_id', 'disease_id']

compound_gene_disease = pd.merge(compound_gene_disease, disease[['@id','name']], left_on='disease_id', right_on='@id', how='inner').drop(['@id'], axis=1)
compound_gene_disease.columns = ['compound_id', 'compound_name', 'gene_id', 'disease_id', 'disease_name']

compound_gene_disease = compound_gene_disease.query('compound_name.str.contains("Valproic Acid")').groupby(['disease_name'],as_index=False).agg({'gene_id': lambda x: x.tolist()})
compound_gene_disease['num_of_genes'] = compound_gene_disease['gene_id'].str.len()

answer_question_2 = compound_gene_disease.sort_values(by=['num_of_genes'], ascending=False).head(n=3)

answer_question_2


## Question 3

In [ ]:
disease_gene1 = pd.merge(disease[['@id','name']], edges_disease_associates[['@id','@id_to']], on='@id')
disease_gene1.columns = ['disease_id', 'disease_name', 'gene_1_id']

disease_gene1_gene_2 = pd.merge(disease_gene1, edges_gene_interacts[['@id', '@id_to']], left_on = 'gene_1_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id']

disease_gene1_gene_2_process = pd.merge(disease_gene1_gene_2, edges_gene_participates[['@id', '@id_to']], left_on = 'gene_2_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2_process.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id', 'process_id']
disease_gene1_gene_2_process = pd.merge(disease_gene1_gene_2_process, biological_process[['@id', 'name']], left_on = 'process_id', right_on = '@id').drop(['@id'], axis=1)
disease_gene1_gene_2_process.columns = ['disease_id', 'disease_name', 'gene_1_id', 'gene_2_id', 'process_id', 'process_name']

answer_question_3 = disease_gene1_gene_2_process.query('disease_name.str.contains("multiple sclerosis") & process_name.str.contains("retina layer formation")')

answer_question_3